# Chroma

> [Chroma](https://docs.trychroma.com/getting-started)는 개발자의 생산성에 초점을 맞춘 AI 기반의 오픈 소스 벡터 데이터베이스입니다. Chroma는 Apache 2.0 라이선스로 제공됩니다.

Chroma를 설치하려면 다음 명령어를 사용하세요


In [ ]:
%pip install -qU chromadb

Chroma는 다양한 모드로 실행될 수 있습니다. 아래에서 각각의 예시를 확인할 수 있습니다.

- `in-memory` - Python 스크립트 또는 Jupyter 노트북에서 실행
- `in-memory with persistance` - 스크립트 또는 노트북에서 실행하고 디스크에 저장/로드
- `in a docker container` - 로컬 머신이나 클라우드에서 서버로 실행

다른 데이터베이스와 마찬가지로 다음과 같은 작업을 수행할 수 있습니다:

- `.add`
- `.get`
- `.update`
- `.upsert`
- `.delete`
- `.peek`
- `.query`: 유사도 검색을 실행합니다.

[참고]

전체 문서는 [docs](https://docs.trychroma.com/reference/Collection)에서 확인할 수 있습니다.

- 이러한 메서드에 직접 액세스하려면 `._collection.method()`를 사용할 수 있습니다.


## 기본 사용법

이 기본 예제에서는 샘플 데이터(appendix-keywords.txt) 를 가져와 청크로 분할하고, 오픈 소스 임베딩 모델을 사용하여 임베딩한 다음, Chroma에 로드하고 쿼리합니다.


- TextLoader를 사용하여 문서를 로드합니다.
- `CharacterTextSplitter`를 사용하여 로드된 문서를 청크 단위로 분할합니다.
- SentenceTransformerEmbeddings를 사용하여 오픈 소스 임베딩 함수를 생성합니다. 모델로는 `"all-MiniLM-L6-v2"`를 사용합니다.
- Chroma 벡터 스토어에 분할된 문서와 임베딩 함수를 로드합니다.
- 질의(query) 를 사용하여 Chroma 벡터 스토어에서 유사도 검색을 수행합니다.
- 검색 결과 중 가장 유사한 문서의 내용을 출력합니다.


In [2]:
# import
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import CharacterTextSplitter

# 문서를 로드하고 청크로 분할합니다.
loader = TextLoader("./data/appendix-keywords.txt")
documents = loader.load()

# 문서를 청크로 분할합니다.
text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

# 오픈 소스 임베딩 함수를 생성합니다.
stf_embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# Chroma에 로드합니다.
db = Chroma.from_documents(docs, stf_embeddings)

# 질의합니다.
query = "What is Word2Vec?"
docs = db.similarity_search(query)

# 결과를 출력합니다.
print(docs[0].page_content)

정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.
예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.
연관키워드: 자연어 처리, 임베딩, 의미론적 유사성
LLM (Large Language Model)


## 로컬 디스크 공간에 저장

이전 예제를 확장하여, 디스크에 저장하려는 경우에는 Chroma 클라이언트를 초기화하고 데이터를 저장할 디렉토리를 전달하기만 하면 됩니다.

[주의]

- Chroma는 데이터를 자동으로 디스크에 저장하기 위해 최선을 다하지만, 여러 개의 인메모리 클라이언트가 서로의 작업을 중단시킬 수 있습니다. 따라서, 한 번에 경로당 하나의 클라이언트만 실행하는 것이 좋습니다.


- `Chroma.from_documents()` 메서드를 사용하여 `docs`와 `embedding_function`으로부터 Chroma 데이터베이스를 생성하고, `persist_directory`를 지정하여 디스크에 저장합니다.
- `db2.similarity_search()` 메서드를 사용하여 `query`와 유사한 문서를 검색합니다.
- `Chroma` 클래스의 생성자를 사용하여 `persist_directory`에 저장된 Chroma 데이터베이스를 로드하고, `embedding_function`을 지정합니다.
- `db3.similarity_search()` 메서드를 사용하여 `query`와 유사한 문서를 검색합니다.
- 검색된 문서 중 첫 번째 문서의 `page_content`를 출력합니다.


In [3]:
# 저장할 경로 지정
DB_PATH = "./chroma_db"

# 문서를 디스크에 저장합니다. 저장시 persist_directory에 저장할 경로를 지정합니다.
db2 = Chroma.from_documents(docs, stf_embeddings, persist_directory=DB_PATH)

# 질의합니다.
query = "What is Word2Vec?"
docs = db2.similarity_search(query)

# 결과를 출력합니다.
docs[0]

Document(page_content='정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.\n예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.\n연관키워드: 자연어 처리, 임베딩, 의미론적 유사성\nLLM (Large Language Model)', metadata={'source': './data/appendix-keywords.txt'})

로컬 디스크에 저장된 Chroma 데이터베이스를 **로드** 한 뒤 쿼리합니다.


In [4]:
# 디스크에서 문서를 로드합니다.
db3 = Chroma(persist_directory="./chroma_db",
             embedding_function=stf_embeddings)

# 질의합니다.
query = "What is Word2Vec?"
docs = db3.similarity_search(query)
print(docs[0].page_content)

정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.
예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.
연관키워드: 자연어 처리, 임베딩, 의미론적 유사성
LLM (Large Language Model)


## Langchain API - Chroma Client 활용

Chroma Client를 생성하여 LangChain에 전달할 수도 있습니다. 이는 기본 데이터베이스에 더 쉽게 액세스하려는 경우에 특히 유용합니다.

또한 LangChain에서 사용할 **collection name** 을 지정할 수 있습니다.


- `chromadb` 모듈을 사용하여 `PersistentClient`를 생성합니다.
- `PersistentClient`를 사용하여 "chroma_collection"이라는 이름의 컬렉션을 가져오거나 생성합니다.
- `collection.add()` 메서드를 사용하여 컬렉션에 문서를 추가합니다. 문서의 ID와 내용을 각각 "1", "2", "3"과 "a", "b", "c"로 지정합니다.
- `Chroma` 클래스를 사용하여 `langchain_chroma` 객체를 생성합니다. 이 객체는 `PersistentClient`, 컬렉션 이름, 임베딩 함수를 매개변수로 받습니다.
- `langchain_chroma._collection.count()` 메서드를 사용하여 컬렉션의 문서 수를 출력합니다.


In [5]:
import chromadb

# ChromaDB의 PersistentClient를 생성합니다.
persistent_client = chromadb.PersistentClient()
# "my_chroma_collection"이라는 이름의 컬렉션을 가져오거나 생성합니다.
collection = persistent_client.get_or_create_collection("chroma_collection")
# 컬렉션에 ID와 문서를 추가합니다.
collection.add(ids=["1", "2", "3"], documents=["a", "b", "c"])

# Chroma 객체를 생성합니다.
langchain_chroma = Chroma(
    # PersistentClient를 전달합니다.
    client=persistent_client,
    # 사용할 컬렉션의 이름을 지정합니다.
    collection_name="chroma_collection",
    # 임베딩 함수를 전달합니다.
    embedding_function=stf_embeddings,
)

# 컬렉션의 항목 수를 출력합니다.
print(
    "현재 저장된 Collection 의 개수는 ",
    langchain_chroma._collection.count(),
    " 개 입니다.",
)

현재 저장된 Collection 의 개수는  3  개 입니다.


## 도커 컨테이너 사용

Chroma Server를 별도의 Docker 컨테이너에서 실행하고, 이에 연결할 Client를 생성한 다음, 이를 LangChain에 전달할 수도 있습니다.

Chroma는 문서의 여러 `Collections`을 처리할 수 있지만, LangChain 인터페이스는 하나의 컬렉션만 처리하므로 컬렉션 이름을 지정해야 합니다.

LangChain에서 사용하는 기본 컬렉션 이름은 `"langchain"` 입니다.


### 도커 이미지를 복제 및 실행


다음은 Docker 이미지를 복제, 빌드 및 실행하는 방법입니다


!git clone git@github.com:chroma-core/chroma.git


`docker-compose.yml` 파일을 편집하고 `environment` 아래에 `ALLOW_RESET=TRUE`를 추가합니다.


```yaml
    ...
    command: uvicorn chromadb.app:app --reload --workers 1 --host 0.0.0.0 --port 8000 --log-config log_config.yml
    environment:
      - IS_PERSISTENT=TRUE
      - ALLOW_RESET=TRUE
    ports:
      - 8000:8000
    ...
```


편집이 완료된 예시

![](./images/chroma-docker1.png)


그런 다음 `docker-compose up -d --build`를 실행합니다.


도커 구동 후 성공적으로 실행되는지 확인합니다.

![](./images/chroma-docker2.png)


`chromadb` 패키지를 사용하여 Chroma 클라이언트를 생성합니다.

- `allow_reset=True`로 설정하여 데이터베이스를 초기화할 수 있도록 합니다.
- `client.reset()`을 호출하여 데이터베이스를 초기화합니다.
- `client.create_collection()`을 사용하여 "chroma_docker"이라는 새로운 컬렉션을 생성합니다.
- `docs` 리스트의 각 문서에 대해 `collection.add()`를 호출하여 고유한 ID, 메타데이터, 문서 내용을 컬렉션에 추가합니다.


In [6]:
# Chroma 클라이언트 생성
import uuid
import random
import chromadb
from chromadb.config import Settings

client = chromadb.HttpClient(
    host="0.0.0.0", port=8000, settings=Settings(allow_reset=True)
)
client.reset()  # 데이터베이스 초기화
collection = client.create_collection("chroma_docker")

# 문서를 로드하고 청크로 분할합니다.
loader = TextLoader("./data/appendix-keywords.txt")
text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0)
docs = loader.load_and_split(text_splitter)

for i, doc in enumerate(docs):
    doc.metadata["chunk_id"] = i  # Chunk ID 추가
    # 무작위로 임의의 페이지 번호를 삽입합니다.
    doc.metadata["page_number"] = random.randint(0, 5)
    collection.add(
        ids=[str(uuid.uuid1())],
        metadatas=doc.metadata,
        documents=doc.page_content,
    )

# LangChain에게 클라이언트와 컬렉션 이름 사용 지시
db4 = Chroma(
    client=client,
    collection_name="chroma_docker",
    embedding_function=stf_embeddings,
)
query = "What is Word2Vec?"
docs = db4.similarity_search(query)
print(docs[0].page_content)

정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.
예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.
연관키워드: 자연어 처리, 임베딩, 의미론적 유사성
LLM (Large Language Model)


## 업데이트 & 삭제

실제 애플리케이션을 구축할 때는 데이터 추가를 넘어 데이터 업데이트와 삭제도 수행하고자 합니다.

Chroma는 이를 단순화하기 위해 사용자가 `ids`를 제공하도록 합니다. `ids`는 파일 이름이나 `filename_paragraphNumber`와 같은 결합된 해시가 될 수 있습니다.

Chroma는 이러한 모든 작업을 지원하지만, 일부는 여전히 LangChain 인터페이스를 통해 완전히 통합되는 과정에 있습니다. 추가적인 워크플로 개선 사항이 곧 추가될 예정입니다.

다음은 다양한 작업을 수행하는 방법을 보여주는 기본 예제입니다:


- 문서 개수에 해당하는 간단한 ID 리스트를 생성합니다.
- `Chroma.from_documents()` 메서드를 사용하여 문서, 임베딩 함수, ID를 전달하고 Chroma 데이터베이스를 생성합니다.
- `similarity_search()` 메서드를 사용하여 쿼리와 유사한 문서를 검색하고, 첫 번째 문서의 메타데이터를 출력합니다.
- 첫 번째 문서의 메타데이터를 업데이트하고, `update_document()` 메서드를 사용하여 데이터베이스에 반영합니다.
- 업데이트된 문서의 정보를 출력합니다.
- 마지막 문서를 삭제하기 전후의 문서 개수를 출력하여 삭제 작업을 확인합니다.


In [7]:
# 간단한 ID 생성
ids = [str(i) for i in range(1, len(docs) + 1)]

# 데이터 추가
example_db = Chroma.from_documents(docs, stf_embeddings, ids=ids)
docs = example_db.similarity_search(query)
print(docs[0].metadata)

# 문서의 메타데이터 업데이트
docs[0].metadata = {
    "source": "./images/appendix-keywords.txt",
    "new_value": "테스트용으로 업데이트할 내용입니다.",
}

# DB 에 업데이트
example_db.update_document(ids[0], docs[0])
print(example_db._collection.get(ids=[ids[0]]))

# 문서 개수 출력
print("count before", example_db._collection.count())
# 마지막 문서 삭제
example_db._collection.delete(ids=[ids[-1]])
# 삭제 후 문서 개수 출력
print("count after", example_db._collection.count())

{'source': './data/appendix-keywords.txt'}
{'ids': ['1'], 'embeddings': None, 'metadatas': [{'chunk_id': 12, 'new_value': '테스트용으로 업데이트할 내용입니다.', 'page_number': 1, 'source': './images/appendix-keywords.txt'}], 'documents': ['정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.\n예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.\n연관키워드: 자연어 처리, 임베딩, 의미론적 유사성\nLLM (Large Language Model)'], 'uris': None, 'data': None}
count before 64
count after 63


## OpenAI Embeddings 사용하기

많은 사람들이 `OpenAIEmbeddings` 를 사용하는 것을 선호합니다.

다음은 `OpenAIEmbeddings` 을 사용한 예제입니다.


In [8]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

- `chromadb.EphemeralClient`를 사용하여 새로운 Chroma 클라이언트를 생성합니다.
- `Chroma.from_documents` 메서드를 사용하여 문서(`docs`)와 임베딩(`embedings`)을 기반으로 Chroma 컬렉션을 생성합니다.
  - 새로 생성된 클라이언트(`new_client`)를 사용합니다.
  - 컬렉션 이름은 "openai_collection"으로 지정합니다.
- 질의(`query`)를 정의합니다: "Word2Vec 에 대해서 설명해 주세요."
- `openai_lc_client.similarity_search` 메서드를 사용하여 질의와 유사한 문서를 검색합니다.


In [10]:
from langchain_openai import OpenAIEmbeddings


# 문서를 로드하고 청크로 분할합니다.
loader = TextLoader("./data/appendix-keywords.txt")
text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0)
docs = loader.load_and_split(text_splitter)

# OpenAIEmbeddings 를 사용
openai_embeddings = OpenAIEmbeddings()
# client 생성
new_client = chromadb.EphemeralClient()


# OpenAI 임베딩과 Chroma 클라이언트를 사용하여 문서에서 Chroma 검색 클라이언트를 생성합니다.
openai_lc_client = Chroma.from_documents(
    docs, openai_embeddings, client=new_client, collection_name="openai_collection"
)

query = "Word2Vec 에 대해서 설명해 주세요."
# 질의를 사용하여 유사도 검색을 수행합니다.
docs = openai_lc_client.similarity_search(query)
# 검색 결과에서 첫 번째 문서의 내용을 출력합니다.
print(docs[0].page_content)

정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.
예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.
연관키워드: 자연어 처리, 임베딩, 의미론적 유사성
LLM (Large Language Model)


### 유사도 검색과 유사도 점수 계산

코사인 유사도를 산정합니다. score가 높을 수록 유사도가 높다고 판단할 수 있습니다.


- `db` 객체의 `similarity_search_with_score` 메서드를 사용하여 `query`와 유사한 문서를 검색하고, 그 결과를 `docs` 변수에 저장합니다.


In [11]:
query = "Word2Vec 에 대해서 설명해 주세요."
# 유사도 검색을 수행하고 점수와 함께 결과를 반환합니다.
docs = db.similarity_search_with_score(query)

- `docs[0]`을 사용하여 문서 리스트의 첫 번째 문서에 접근합니다.


In [12]:
# 문서 리스트의 첫 번째 문서를 가져옵니다.
document, score = docs[0]
print(document, end="\n\n")
print(f"유사도 점수: {score}")

page_content='정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.\n예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.\n연관키워드: 자연어 처리, 임베딩, 의미론적 유사성\nLLM (Large Language Model)' metadata={'source': './data/appendix-keywords.txt'}

유사도 점수: 0.8870154023170471


### 검색기(Retriever)

이 섹션에서는 Chroma를 retriever로 사용하는 다양한 옵션에 대해 설명합니다.

### MMR

retriever 객체에서 유사도 검색을 사용하는 것 외에도 `mmr`을 사용할 수 있습니다.


- `db` 객체를 사용하여 `retriever`를 생성합니다.
- `search_type` 매개변수를 "mmr"로 설정하여 MMR(Maximal Marginal Relevance) 검색 알고리즘을 사용합니다.


In [13]:
# 검색 유형을 "mmr"로 설정하여 데이터베이스를 검색기로 사용
retriever = db.as_retriever(search_type="mmr")

- `retriever` 객체의 `get_relevant_documents()` 메서드를 사용하여 주어진 `query`와 관련된 문서를 검색합니다.
- `get_relevant_documents()` 메서드는 관련 문서의 리스트를 반환하며, 이 중 첫 번째 문서([0])를 선택합니다.


In [15]:
query = "Word2Vec 에 대해서 설명해 주세요."
# 질의와 관련된 문서 중 가장 관련성이 높은 문서를 가져옵니다.
retriever.get_relevant_documents(query)[0]

Document(page_content='정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.\n예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.\n연관키워드: 자연어 처리, 임베딩, 의미론적 유사성\nLLM (Large Language Model)', metadata={'source': './data/appendix-keywords.txt'})

## 메타데이터를 기반으로 필터링

컬렉션을 다루기 전에 컬렉션의 범위를 좁히는 것이 도움이 될 수 있습니다.

예를 들어, `get` 메서드를 사용하여 메타데이터를 기반으로 컬렉션을 필터링할 수 있습니다.


- `example_db.get()` 메서드를 사용하여 메타데이터 필터링을 해보겠습니다.


In [31]:
# metadatas 중 chunk_id 기준으로 필터링
example_db.get(where={"chunk_id": 0})

{'ids': ['2'],
 'embeddings': None,
 'metadatas': [{'chunk_id': 0,
   'page_number': 0,
   'source': './data/appendix-keywords.txt'}],
 'documents': ['Semantic Search\n\n정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.\n예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.\n연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝\n\nEmbedding'],
 'uris': None,
 'data': None}

In [27]:
# page_number 기준으로 필터링
example_db.get(where={"page_number": 0})

{'ids': ['2'],
 'embeddings': None,
 'metadatas': [{'chunk_id': 0,
   'page_number': 0,
   'source': './data/appendix-keywords.txt'}],
 'documents': ['Semantic Search\n\n정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.\n예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.\n연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝\n\nEmbedding'],
 'uris': None,
 'data': None}

In [32]:
# source 기준으로 필터링
example_db.get(where={"source": "./data/appendix-keywords.txt"})

{'ids': ['038f8d1a-3791-4478-a1f3-270ee38f6198',
  '04d5339d-2a5b-43b6-8e27-4ef618912aeb',
  '07625557-455a-48e7-ac03-58a6836ec615',
  '0782f1c3-5dd7-425e-bf6a-664dceac3dba',
  '07eebb17-9d6f-4cc7-9808-fa6d9acb664f',
  '17d5bd29-e616-4d2f-9fe5-b66d71e8f9e1',
  '2',
  '24036367-5369-479a-a453-545f997dcf5f',
  '25df25a0-69d1-4d64-812b-40f04adcc9e2',
  '2601bff8-44da-4085-be61-60b7de6cfbb4',
  '26edcd5c-cf40-4ac5-a45d-2583973b77f1',
  '28293504-59aa-4081-ae40-c3b36efd2180',
  '3',
  '34515af0-4aa8-4833-9871-7baf17e27e9a',
  '37bc89ef-22e2-4e72-8103-452b658c8827',
  '3b9320f4-cfa9-44b0-85a5-18cf08eb7b0c',
  '3f4db244-f470-4f29-ba38-e6c842739040',
  '402ab127-b6a6-4bb7-994f-dd2027a6d4a7',
  '478cab49-e4ee-42d5-936c-971bef2770d2',
  '49c5f021-87cf-4eaa-965d-2406f48d19ba',
  '49cba4ff-1fe2-43ef-92c5-e1a914ff8411',
  '4a2f8c04-3b1a-48a1-9167-6c9423d49e29',
  '4f502b57-a713-4c44-a32e-76d83aaabccc',
  '53aff5c4-100b-41bf-83a3-0df92df7e216',
  '563255f4-310b-424b-9b8c-2a09828cbec2',
  '59af4f5b-5